In [6]:
import urllib.request
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urltools
import time 


# In[2]:
#location="`D:\WHIN\"

def get_base_url(url):
    from urllib.parse import urlsplit
    base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url))
    

    if base_url.startswith("https"):
        base_url = base_url.replace('https://','')
    elif base_url.startswith("http"):
        base_url = base_url.replace('http://','')
        
    if base_url.endswith('/'):
        base_url = base_url.replace('/','')
    return base_url


# In[3]:


def does_page_exist(url):
    exists = 0 
    try:
        request = requests.get(url)
        if request.status_code == 200:
            exists = 1 
            
    except: 
        pass
    
    return exists


# In[4]:


# Function to extract links from a URL------------------------------------------
def href_scrapper(url):

    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        a_tag = soup.find_all('a')
        img_tag = soup.find_all('img')
        link_tag = soup.find_all('link')
        link_list = list()

        for links in a_tag:
            link_list.append(links.get('href'))

        for links in link_tag:
            link_list.append(links.get('href'))

        for links in img_tag:
            link_list.append(links.get('src'))

        return link_list

    except:
        return 0



# In[5]:


# Function to create complete links---------------------------------------------
def link_former(url):
    x = url

    if x.startswith("http"):
        return x

    else:
        full_url = original_url + x
        return full_url



# In[6]:


# Function to save links to directory of python file----------------------------
def file_save(url, i):
    save_error = 0
    full_url = url
    try:
        if str(full_url).endswith(".htm"):
            name = "WebPage" + str(i) + ".htm"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".pdf"):
            name = "Pdf" + str(i) + ".pdf"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".jpg"):
            name = "Image" + str(i) + ".jpg"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".jpeg"):
            name = "Image" + str(i) + ".jpeg"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".png"):
            name = "Image" + str(i) + ".png"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".css"):
            name = "CSS File" + str(i) + ".css"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        else:
            name = "WebPage" + str(i) + ".html"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        return m
    except:
        save_error = save_error + 1




# In[7]:

def crawl(original_url, num_Id):
    
    # Accepting input of URL and depth----------------------------------------------
    tic = time.time()
    
    parent_list = [original_url]
    url_to_check = get_base_url(original_url)
    print(url_to_check)
    url_to_check = str(url_to_check)

    layer_stop = 1
    layer = 0


    #initializing all the requiered lists---------------------------------------------
    visited_all  = [original_url]
    visited_current_layer = []
    child_list =[]
    child_list_filtered = []
    #columns = ['Link','Parent Link', 'Layer']
    df = pd.DataFrame()


    # Main execution of scrapper----------------------------------------------------

    #looping through layers 
    while layer < layer_stop:
    
        #looping through URLs in parent-list
        for url in parent_list:
        
            #scraping the children from the parent url----------------------------
            if href_scrapper(url) != 0:
                child_list = href_scrapper(url)
    
    
            for child in child_list:
                if child != None:
                    #if child link is of the form "index.php/blahblah" and parent ends with '/'
                    #---> "parentlink/index.php/blahblah"
                    if child.startswith('/'):
                        child= str(url) + str(child)
                    
                    if url.endswith('/') and url_to_check not in child:
                        child = str(url) + str(child)
                    
                #normalize the child links-------------------------------------
                    child=urltools.normalize(child)  
                    

                    #filtering out based on 1) External 2) Repeating 3) Invalid links---------------------------
                    if url_to_check in child and child not in visited_all and does_page_exist(child)==1:
                        child_list_filtered.append(child)
                
                    #adding everthing to visited all--------------------
                    if child not in visited_all:
                        child_slash = child + '/'
                        visited_all.append(child)  
                        visited_all.append(child_slash)
                    
                    
            #adding  the visited and filtered children into the "current visited layer" ------ 
            for child_filtered in child_list_filtered:
                visited_current_layer.append(child_filtered)

            #creating a Pandas dataframe to store everything for download----------
            layer_number = [layer+1]*len(child_list_filtered)
            parent_of_child = [url]*len(child_list_filtered)

            df_child = pd.DataFrame(child_list_filtered)
            df_parent = pd.DataFrame(parent_of_child)
            df_layer = pd.DataFrame(layer_number)


            df_to_be_added = pd.concat([df_child,df_parent,df_layer], axis=1)
            df = pd.concat([df,df_to_be_added],ignore_index=True, axis = 0)
            #----------------------------------------------------------------------
        
            #emptying the child lists
            child_list = []
            child_list_filtered = []
        
        #condition to stop filtering-----------------------------------------------
        if not visited_current_layer :
            layer_stop = layer_stop 
        else:
            layer_stop += 1

    
        #child layer is now parent layer--------------------------------------------
        parent_list = []
    
        #we only dont add .png, .jpg , .pdf to the new parent layer 
        for visited_current in visited_current_layer: 
            print(visited_current)
            if(not visited_current.endswith('.png') and not visited_current.endswith('.jpg') and not  visited_current.endswith('.pdf')):
                parent_list.append(visited_current)
            
            
        #displaying the links in different layers----------------------------------
        #print("Links in LAYER:" + str(layer+1))
        print("No of links = " + str(len(visited_current_layer)))
        #print(visited_current_layer)
        print("\n")
        visited_current_layer = [] 
        #updating the layer number
        layer +=1
    file_name = 'C:/Users/mihirbhatia999/Desktop/DCMME/scraped_data/'
    df.to_csv(file_name + str(num_Id) + '.csv', sep=',', encoding='utf-8')
    
    return df, num_Id


In [7]:
import requests

page = requests.get("https://docs.python.org/3/howto/urllib2.html")
from bs4 import BeautifulSoup
from bs4.element import Comment
soup = BeautifulSoup(page.content, 'html.parser')


#information based on:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True
p_tag=soup.find_all('p')

#extract text from html: 
def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)
web_text = []


In [10]:
import json
import urllib
import os 
os.chdir("C:/Users/mihirbhatia999/Desktop/DCMME/scraped_data/")
def scrape(df, num_Id):
    TempD={}
    for index,item in df.iterrows():
        try:      
            print(item[1])
            html = urllib.request.urlopen(str(item[1])).read()
            web_text.append(text_from_html(html))
            print(text_from_html(html))
            TempD[item[1]]=text_from_html(html)
        except: 
            pass
    with open('json'+str(num_Id)+'.json', 'w') as outfile:  
        json.dump(TempD, outfile)
    return 1      

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: '\u202aD://DCMME/scraped_data'

In [ ]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
excel_file = 'C:/Users/mihirbhatia999/Desktop/DCMME/scraped_data/trial1.xlsx'
df = pd.read_excel(excel_file , sheet_name='From List of Companies')
df


In [11]:
#this is the code to run everything at once: 
i = 1 
for index,row in df.iterrows():
    num_Id=row[0]
    original_url=row[2]
    try:
        websites= pd.read_csv(row[0] + '.csv')
    except:
        websites, num_Id = crawl(row[2],row[0])
        filename = "website" + str(row[0]) + ".csv"
        df_of_child_links = pd.read_csv(filename)
        df_of_child_links
        flag = scrape(df_of_child_links ,i)  
    i = i + 1 

www.mihirbhatia.com
https://www.mihirbhatia.com/s/curriculum-vitae-4.pdf
https://www.mihirbhatia.com/bio/
https://www.mihirbhatia.com/
https://www.mihirbhatia.com/experience/
https://www.mihirbhatia.com/projects/
No of links = 5


No of links = 0


https://www.mihirbhatia.com/s/curriculum-vitae-4.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


�4������c'�pD�f6�u8"�����pD��K��p<�������U_$jf3Y�#u�,2����f�Gj�	b��[�Ge��XbĤA:k��'^u8�n5���#F\U��٭�.��_e���~I:C��4H�c�V�cH��bĤA:��;~�G$jf3Y�#5�������Qd�;z�G:�4����F�f6�u8"Q3��:���6�u8"Q3��:��٤A:��{PEy��X�#܏��G$ڻ4�G$ꎟ��ё���8�p��~ �u8"���E��D�|&�pD��!��D�|&�p<���Ҩ:�qiTW���{qT�3i��f� ����l&�p�1b� �%FL�ñv���d�-F��t8"Q�q�>�\���Jk�aH�#5��p��ܵ<�����L��D�l&�pD��!��D'c(2����f�G:�4���^��:��D�l&�p�j`wiX�ሿ��L���Ҭ��>����L�ñ�K�Y�c�FҎ����E���u�.���:��^��:���l[���
�s<��z�m��s'=�賧�(��e�Ӗ��?�ڷٯ�ҿl+ښ�9�v*�_ګ��v��>�sj��K����f�4���x�-��J�W'�����n��xpx�S��@����!ŋ���@�M^��F��2^�A����`Q�1�k�U�����W�xH�v�{v����ݏv�=��'c��Z�2�(�m�֊��7�n��M{�ڛ~m��9{�έ0~!��j�7ث�7a�:˽�W!��س0i���=뵟n?y:��q�W���Ma��v0�_�q��ߗ�G��6|�OE:?�i
�q��T5uɮ��y�Ii�nJ{�
�6�]��Z�j.��
����;�M�ؽ�nn=�����>�����-^��Lg{��j1��g4�-�c���G6{���m�+�dے�Ҿ��q�
_��g��"�.C�9�*�ȭ�P�YM��
/r�.T|�So�⋜:_��W(�*��B�9�*�ȭ�P�YM��
/r�&T;|�S/�⋜:	_��G��"�.B���
��W�����\

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://www.mihirbhatia.com/bio/
https://www.mihirbhatia.com/
            RESUME  Bio  HOME  EXPERIENCE  PROJECTS      RESUME  Bio  HOME  EXPERIENCE  PROJECTS                                                         ASPIRING DATA SCIENTIST | RESEARCHER | MECHANICAL ENGINEER              AVIATOR LONDON         
https://www.mihirbhatia.com/experience/
            RESUME  Bio  HOME  EXPERIENCE  PROJECTS      RESUME  Bio  HOME  EXPERIENCE  PROJECTS                                                         ASPIRING DATA SCIENTIST | RESEARCHER | MECHANICAL ENGINEER                  Data Science Summer Intern --- JUNE 2018 - Present As a data science intern, I'm working on a project that involves development of a dimensional database & data warehousing for a hospital using Amazon Web Services and SQL Server. We will then develop predictive models to allow hospitals to predict the resource requirements, patient inflow and outflow which can make them efficient in their scheduling/buying practices.

https://www.ajintom.com/
https://www.ajintom.com/bio/
https://www.ajintom.com/music/
https://www.ajintom.com/tech#link_es
No of links = 4


https://www.ajintom.com/tech#link_es/tech/
https://www.ajintom.com/tech#link_es/tech#link_mt
https://www.ajintom.com/tech#link_es/tech#link_cn
https://www.ajintom.com/tech#link_es/home
https://www.ajintom.com/tech#link_es/s/Project_Synopsis.pdf
No of links = 5


https://www.ajintom.com/tech#link_es/tech#link_mt/bio/
https://www.ajintom.com/tech#link_es/tech#link_mt/music/
https://www.ajintom.com/tech#link_es/tech#link_mt/s/BLE_IPS_Poster.pdf
No of links = 3


No of links = 0


https://www.ajintom.com/
https://www.ajintom.com/bio/
            Home  Bio  Music     Research & Projects     Music Technology    Communication Networks    Embedded Systems       Resume  Contact      Home  Bio  Music     Research & Projects     Music Technology    Communication Networks    Embedded Systems       Resume  Contact                                             Aji

https://www.ajintom.com/tech#link_es/tech#link_cn
            Home  Bio  Music     Research & Projects     Music Technology    Communication Networks    Embedded Systems       Resume  Contact      Home  Bio  Music     Research & Projects     Music Technology    Communication Networks    Embedded Systems       Resume  Contact                                             Ajin Tom   Technology Researcher | Sound Designer | Composer | Pianist          Research & Projects   Music Technology    Communication Networks    Embedded Systems  Music Technology This page consists of my highlight music technology projects I built over the last few years. Majority of the projects involve Hyper-Instruments design based on MIDI, audio signal processing based projects and digital & analog music visualizers(few signal processing based, and some completely hardware). Built this Digital Musical Instrument (interpretation of 'The Sponge') out of a block of foam, fit with several sensors, to perform some of m

https://www.ajintom.com/tech#link_es/s/Project_Synopsis.pdf
https://www.ajintom.com/tech#link_es/tech#link_mt/bio/
https://www.ajintom.com/tech#link_es/tech#link_mt/music/
https://www.ajintom.com/tech#link_es/tech#link_mt/s/BLE_IPS_Poster.pdf


In [16]:
import json
from pprint import pprint

with open('json1.json') as f1:
    data1 = json.load(f1)
    
with open('json2.json') as f2:
    data2 = json.load(f2)
    



In [17]:
pprint(data1)

{'https://www.mihirbhatia.com/': '            RESUME  Bio  HOME  EXPERIENCE  '
                                 'PROJECTS      RESUME  Bio  HOME  EXPERIENCE  '
                                 'PROJECTS                                                         '
                                 'ASPIRING DATA SCIENTIST | RESEARCHER | '
                                 'MECHANICAL ENGINEER              AVIATOR '
                                 'LONDON         ',
 'https://www.mihirbhatia.com/experience/': '            RESUME  Bio  HOME  '
                                            'EXPERIENCE  PROJECTS      RESUME  '
                                            'Bio  HOME  EXPERIENCE  '
                                            'PROJECTS                                                         '
                                            'ASPIRING DATA SCIENTIST | '
                                            'RESEARCHER | MECHANICAL '
                                            'ENGIN

In [18]:
pprint(data2)

{'https://www.ajintom.com/bio/': '            Home  Bio  Music     Research & '
                                 'Projects     Music Technology    '
                                 'Communication Networks    Embedded '
                                 'Systems       Resume  Contact      Home  '
                                 'Bio  Music     Research & Projects     Music '
                                 'Technology    Communication Networks    '
                                 'Embedded Systems       Resume  '
                                 'Contact                                             '
                                 'Ajin Tom   Technology Researcher | Sound '
                                 'Designer | Composer | Pianist          I am '
                                 'a first year graduate student in Music '
                                 'Technology at Schulich School of Music, '
                                 'McGill University, Canada. I completed my '
     